# WeatherPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint

# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)


## Generate cities list

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=150)
lngs = np.random.uniform(low=-180.000, high=180.000, size=150)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

113

### Perform API calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).

In [22]:
# Create a base url
base_url = "http://api.openweathermap.org/data/2.5/weather?units=imperial&appid=" + api_key

# Counters
city_counter = 1
set_counter = 1

# Create the lists to hold relative data
cloudiness = []
country = []
date = []
humidity = []
lat = []
lng = []
max_temp = []
wind_speed = []

print("Started")

# Create a query url for each city in the cities list to get json response
for i, city in enumerate(cities):
    
    # Group cities as sets of 50s
    if (i % 50 == 0 and i >= 50):
        set_counter += 1
        city_counter = 1
         
    # Create API url for each city
    query_url = base_url + "&q=" + city
    
    # print(f"url:", query_url)
    
    # Print the results 
    print("Processing Record %s of Set %s | %s" % (city_counter, set_counter, city))
    
    # Make API request for each city
    try:
        # Increase city counter
        city_counter += 1
    
        # Perform a request for data on each city
        response = requests.get(query_url).json()
        
        # print(f"response:", response)
        
        cloudiness.append(response["clouds"]["all"])
        country.append(response["sys"]["country"])
        date.append(response["dt"])
        humidity.append(response["main"]["humidity"])
        lat.append(response["coord"]["lat"])
        lng.append(response["coord"]["lon"])
        max_temp.append(response["main"]["temp_max"])
        wind_speed.append(response["wind"]["speed"])
        
        weather_data = {
            "City": cities,
            "Cloudiness": cloudiness,
            "Country": country,
            "Date": date,
            "Humidity": humidity,
            "Lat": lat,
            "Lng": lng,
            "Max Temp": max_temp,
            "Wind Speed": wind_speed    
        }
    
    except:
        
        print("City not found. Skipping...")
        pass
        
print("Completed")

Started
Processing Record 1 of Set 1 | bolungarvik
City not found. Skipping...
Processing Record 2 of Set 1 | kahului
Processing Record 3 of Set 1 | turayf
Processing Record 4 of Set 1 | vaitupu
City not found. Skipping...
Processing Record 5 of Set 1 | porbandar
Processing Record 6 of Set 1 | padang
Processing Record 7 of Set 1 | hermanus
Processing Record 8 of Set 1 | mosquera
Processing Record 9 of Set 1 | hays
Processing Record 10 of Set 1 | san carlos del zulia
Processing Record 11 of Set 1 | punta arenas
Processing Record 12 of Set 1 | rikitea
Processing Record 13 of Set 1 | avarua
Processing Record 14 of Set 1 | qaanaaq
Processing Record 15 of Set 1 | olinda
Processing Record 16 of Set 1 | jamestown
Processing Record 17 of Set 1 | mataura
Processing Record 18 of Set 1 | chapais
Processing Record 19 of Set 1 | eyl
Processing Record 20 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 21 of Set 1 | lolua
City not found. Skipping...
Processing Record 22 of Set 1 

{'base': 'stations',
 'clouds': {'all': 90},
 'cod': 200,
 'coord': {'lat': 43.8, 'lon': -91.24},
 'dt': 1569619107,
 'id': 5258957,
 'main': {'humidity': 100,
          'pressure': 1006,
          'temp': 66.42,
          'temp_max': 71.6,
          'temp_min': 62.01},
 'name': 'La Crosse',
 'sys': {'country': 'US',
         'id': 6319,
         'message': 0.0065,
         'sunrise': 1569585454,
         'sunset': 1569628462,
         'type': 1},
 'timezone': -18000,
 'visibility': 16093,
 'weather': [{'description': 'overcast clouds',
              'icon': '04d',
              'id': 804,
              'main': 'Clouds'}],
 'wind': {'deg': 250, 'speed': 4.7}}


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [24]:
len(cities)
#len(lat)

113

In [25]:
# Create the dictionary to keep data 
# weather_data = {
#     "City": cities,
#     "Cloudiness": cloudiness,
#     "Country": country,
#     "Date": date,
#     "Humidity": humidity,
#     "Lat": lat,
#     "Lng": lng,
#     "Max Temp": max_temp,
#     "Wind Speed": wind_speed    
# }

#weather_df.count()

# Create the data frame
weather_df = pd.DataFrame(weather_data)
weather_df.head()

ValueError: arrays must all be same length

In [9]:
weather_df.count()

NameError: name 'weather_df' is not defined

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot